In [ ]:
# 1_load_dataset.py

import pandas as pd

def load_dataset(csv_file="public_services_asr_manifest.csv"):
    df = pd.read_csv(csv_file)
    print("✅ Dataset loaded:", df.shape)
    print(df.head(5))
    return df

if __name__ == "__main__":
    df = load_dataset()


In [ ]:
# 2_preprocess_clean.py

import pandas as pd

def clean_dataset(df):
    df.dropna(subset=["transcription", "audio_filepath"], inplace=True)
    df.drop_duplicates(subset=["language", "transcription"], inplace=True)
    print("✅ After cleaning:", df.shape)
    return df

if __name__ == "__main__":
    df = load_dataset()
    df = clean_dataset(df)
    print(df.head(5))


In [ ]:
# 3_preprocess_normalize.py

import unicodedata
import re
def normalize_text(text):
    text = str(text)
    text = unicodedata.normalize("NFC", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def normalize_dataset(df):
    df["transcription"] = df["transcription"].map(normalize_text)
    print("✅ After normalization sample:")
    print(df[["language", "transcription"]].head(5))
    return df

if __name__ == "__main__":
    df = load_dataset()
    df = clean_dataset(df)
    df = normalize_dataset(df)


In [ ]:
# 4_preprocess_split.py

import json
from pathlib import Path
from sklearn.model_selection import train_test_split

def split_dataset(df):
    train, temp = train_test_split(df, test_size=0.2, stratify=df["language"], random_state=42)
    val, test = train_test_split(temp, test_size=0.5, stratify=temp["language"], random_state=42)

    print("✅ Train size:", len(train))
    print("✅ Validation size:", len(val))
    print("✅ Test size:", len(test))
    return train, val, test

def save_manifests(train, val, test, output_dir="manifests"):
    Path(output_dir).mkdir(exist_ok=True)
    for split_name, split_df in {"train": train, "validation": val, "test": test}.items():
        out_path = Path(output_dir) / f"{split_name}_manifest.jsonl"
        with open(out_path, "w", encoding="utf-8") as f:
            for _, row in split_df.iterrows():
                f.write(json.dumps({
                    "audio_filepath": row["audio_filepath"],
                    "transcription": row["transcription"],
                    "language": row["language"]
                }, ensure_ascii=False) + "\n")
        print(f"✅ {split_name} manifest saved at {out_path}")

if __name__ == "__main__":
    df = load_dataset()
    df = clean_dataset(df)
    df = normalize_dataset(df)
    train, val, test = split_dataset(df)
    save_manifests(train, val, test)
